In [ ]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Lambda,Concatenate
from tensorflow.keras import optimizers, regularizers, applications
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model

import numpy as np
import scipy.io as sio
import scipy.stats as sst
import scipy

import random
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, roc_auc_score, average_precision_score

import matplotlib.pyplot as plt

import keras

In [ ]:
monitor_model = models.load_model('./pre_trained_model_D1.h5')

In [ ]:
def monitor_evaluation(X,Y,model):
    predict = (model.predict(X)>0.5).reshape([-1])
    cm = confusion_matrix((Y>0.5).reshape([-1]),predict);print(cm/1000)
    print("predict accuracy",np.mean(predict>0.5))
    print("true accuracy",np.mean(Y>0.5))

In [ ]:
# test it on train
## load encode feature, prob and tcp
monitor_data = sio.loadmat('./D1_test_features.mat')#cifar_test; clinic10_train; clinic10_test;
monitor_test_X = monitor_data['probability']
label = monitor_data['label'].reshape([-1])
prediction = monitor_data['prediction'].reshape([-1])
monitor_test_Y = label!=prediction

# get monitor_test_tcp
monitor_test_tcp = np.zeros((monitor_test_X.shape[0]))
for i in range(monitor_test_X.shape[0]):
    monitor_test_tcp[i]=monitor_test_X[i,int(label[i])]
# monitor_test_tcp = monitor_data['true_class_prob'].reshape([-1])
print(monitor_test_X.shape, monitor_test_Y.shape,monitor_test_tcp.shape)
# monitor_evaluation(monitor_test_X,1-monitor_test_Y,monitor_model)

In [ ]:
# filter out the nan rows: monitor_test_X, monitor_test_tcp, monitor_test_Y
monitor_test_X_new = np.zeros((0,monitor_test_X.shape[1]))
monitor_test_tcp_new = np.zeros((0,))
monitor_test_Y_new = np.zeros((0,))

for i in range(monitor_test_X.shape[0]):
    filter_1 = monitor_test_X[i];
    filter_2 = np.sum(np.isnan(filter_1))
    if(filter_2==0):
        monitor_test_X_new = np.vstack((monitor_test_X_new,monitor_test_X[i]))
        monitor_test_tcp_new = np.hstack((monitor_test_tcp_new,monitor_test_tcp[i]))
        monitor_test_Y_new = np.hstack((monitor_test_Y_new,monitor_test_Y[i]))

In [ ]:
## transfer data
## type 1: active
# monitor_model = models.load_model('monitor_without_image_v2.h5')
# predict_TCP = monitor_model.predict(monitor_test_X[:,-10:])
# TCP_entrop = predict_TCP*(1-predict_TCP)*4
# transfer_list = TCP_entrop.reshape([-1]).argsort()[::-1][:90]#[np.random.choice(9000,900)]
# monitorX_train = monitor_test_X[transfer_list]
# monitorY_train = monitor_test_tcp[transfer_list]

## type 2: not active
monitorX_train,monitorX_test,monitorY_train,monitorY_test = train_test_split(monitor_test_X_new,monitor_test_tcp_new,
                                                                             test_size=0.99,random_state=42)
print(monitorX_train.shape, monitorY_train.shape)

In [ ]:
## transfer block
monitor_model = models.load_model('pre_trained_model_D1.h5')
monitor_evaluation(monitor_test_X_new,1-monitor_test_Y_new,monitor_model)

In [ ]:
## transfer block
monitor_model = models.load_model('pre_trained_model_D1.h5')
for layer in monitor_model.layers:
    layer.trainable=False
# monitor_model.layers[-3].trainable=True
# monitor_model.layers[-2].trainable=True
monitor_model.layers[-1].trainable=True

monitor_model.compile(optimizer='Adam',loss='mean_squared_error')
## transfer 
history = monitor_model.fit(monitorX_train,monitorY_train,epochs=250,shuffle=True)

In [ ]:
monitor_evaluation(monitor_test_X_new,1-monitor_test_Y_new,monitor_model)

In [ ]:
pp = monitor_model.predict(monitor_test_X_new).reshape([-1])

roc_auc_score(monitor_test_Y_new,1-pp), average_precision_score(monitor_test_Y_new,1-pp)

In [ ]:
pp = np.max(monitor_test_X_new,axis=-1)

roc_auc_score(monitor_test_Y_new,1-pp),average_precision_score(monitor_test_Y_new,1-pp)

In [ ]:
models.save_model(monitor_model,'pre_trained_model_D1_transferred.h5')